# Homework Week 3

## Question 1

In [1]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [3]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)

In [4]:
v[0]

0.078222655

**Answer = 0.07**

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [7]:
len(documents)

948

In [8]:
documents = [item for item in documents if item['course'] == 'machine-learning-zoomcamp']
len(documents)

375

## Question 2

In [9]:
import numpy as np
from tqdm.auto import tqdm

In [10]:
embeddings = []

for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    qa_text_embedding = embedding_model.encode(qa_text)
    embeddings.append(qa_text_embedding)

X = np.array(embeddings)

  0%|          | 0/375 [00:00<?, ?it/s]

In [11]:
print(X.shape)

(375, 768)


**Answer = (375, 768)**

## Question 3

In [12]:
scores = X.dot(v)

In [13]:
scores.max()

0.6506573

**Answer = 0.65**

In [14]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Question 4

In [15]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [16]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_engine.search(embedding_model.encode(q['question']), num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [17]:
def hit_rate(relevance_total):
    cnt = 0
    
    for line in relevance_total:
        if True in line:
            cnt = cnt + 1
            
    return cnt / len(relevance_total)

In [18]:
print(hit_rate(relevance_total))

0.9398907103825137


**Answer = 0.93**

## Question 5

In [19]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "qa_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
     }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [20]:
for doc in tqdm(documents):
    doc['qa_text_vector'] = embedding_model.encode(doc['question'] + ' ' + doc['text'])

  0%|          | 0/375 [00:00<?, ?it/s]

In [21]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [22]:
def elastic_search_knn(vector):
    knn = {
        "field": "qa_text_vector",
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": "machine-learning-zoomcamp"
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [23]:
search_res = elastic_search_knn(v)
search_res[0]

{'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'section': 'General course-related questions',
 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'id': 'ee58a693'}

**Answer = ee58a693**

## Question 6

In [24]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    v = embedding_model.encode(q['question'])
    results = elastic_search_knn(v)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

print(hit_rate(relevance_total))

  0%|          | 0/1830 [00:00<?, ?it/s]

0.9398907103825137


**Answer = 0.93**